#### Set the input parameters

In [389]:
center_lat =  42.692688 # latitude of center point, in decimal degrees
center_lon =  -72.513084 # longitude of center point, in decimal degrees
radius = 100 # radius of flower of life, in kilometres
divisions = 6 # creates flower of life by dividing the 360 degree circle into 6 equal parts
rotation = 0 # optional: add a tilt to the grid. I do not recommend tilting your grids
max_points = 10000 # the number of points you want the grid to be. ideally keep this below 500

#### Logic for calculating distances between points and creating pattern

In [390]:
from math import asin, atan2, cos, degrees, radians, sin
import numpy as np
import pandas as pd
from google.colab import files
import math

# R = 6371

def get_point_at_distance(lat1, lon1, d, bearing, R=6371.0088):
    """
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2),)



In [391]:
# up with iteration_number
# bottom left for iteration_number
# bottom for iteration_number
# bottom right for iteration_number
# top right for iteration_number
# top for iteration_number
# top left for iteration_number
# then use start_point as value from cut[iteration_number] if i in cut

cuts = []

tmp_cut = 1

for i in range(1, 2^1000000):
  tmp_cut += divisions*i
  cuts.append(tmp_cut)
  if tmp_cut > max_points:
    break

results_df = pd.DataFrame({'lat':[center_lat],
                              'lon':[center_lon]})

max_iteration_number = len(cuts)

for iteration_number in range(1, max_iteration_number+1):


  if iteration_number >= 3:
    start_point = results_df.loc[cuts[iteration_number-3]]

  elif iteration_number == 2:
    start_point = results_df.loc[1]

  else:
    start_point = results_df.loc[0]

  # print(iteration_number, 'start_point', start_point)

  bearing = [0]

  for j in range(0, iteration_number):
    bearing.append(240)

  for j in range(0, iteration_number):
    bearing.append(180)

  for j in range(0, iteration_number):
    bearing.append(120)

  for j in range(0, iteration_number):
    bearing.append(60)

  for j in range(0, iteration_number):
    bearing.append(0)

  for j in range(0, iteration_number-1):
    bearing.append(300)

  # print(bearing)

  for k in bearing:
    tmp_result_lat, tmp_result_lon = get_point_at_distance(start_point['lat'], start_point['lon'], radius, k)
    df_tmp_coord = pd.DataFrame({'lat':[tmp_result_lat], 'lon':[tmp_result_lon]})

    if len(results_df) < max_points:

      results_df = pd.concat([results_df, df_tmp_coord])
      results_df = results_df.reset_index(drop=True)
      start_point = results_df.tail(1)

    else:
      break


  # print(results_df)


<ipython-input-390-f62bb0a08cc0>:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lat1 = radians(lat1)
<ipython-input-390-f62bb0a08cc0>:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lon1 = radians(lon1)


#### Creation of Output file. Import it into Google Earth by doing File -> Import and clicking on output.txt

In [392]:
 # pastes the resulting coordinates into a text file to import into Google Earth
output_text = ''
output_text += 'latitude'+','+'longitude'
output_text += '\n'
for i,r in results_df.iterrows():
  tmp_pt = str(r['lat'])+','+str(r['lon'])+'\n'
  output_text += tmp_pt

f = open("output.txt", "w")
f.write(output_text)
files.download('output.txt')
f.close()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>